In [93]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import warnings;warnings.filterwarnings('ignore')
import re

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics


pd.options.mode.copy_on_write = True
plt.rcParams['font.family'] ='AppleGothic'
plt.rcParams['axes.unicode_minus'] =False

# 데이터를 load (역세권 여부 추가한 데이터를 불러오겠습니다.)
dt = pd.read_csv('../../modified_data/final.csv')
subway = pd.read_csv('../../data/subway_feature.csv')

# 전처리 편의를 위해 컬럼명 조정
subway = subway.rename(columns={'위도': '좌표Y', '경도': '좌표X'})

In [94]:

# 하나의 데이터로 만들어줍니다.
dt['is_test'] = 0
dt.loc[dt['target'].isna(), 'is_test'] = 1

In [95]:
dt['등기신청일자'] = dt['등기신청일자'].replace(' ', np.nan)
dt['거래유형'] = dt['거래유형'].replace('-', np.nan)
dt['중개사소재지'] = dt['중개사소재지'].replace('-', np.nan)

In [96]:
selected = list(dt.columns[dt.isnull().sum() <= 1000000])
concat_select = dt[selected]

In [97]:
concat_select.head(1)

,Unnamed: 0,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,...,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,최상층,is_test
0,0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,202307,26,5,...,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,NaN,5.0,1


In [98]:
concat_select.isna().sum()

Unnamed: 0                     0
시군구                            0
번지                           227
본번                            75
부번                            75
아파트명                        2136
전용면적(㎡)                        0
계약년월                           0
계약일                            0
층                              0
건축년도                           0
도로명                            0
k-단지분류(아파트,주상복합등등)        877273
k-전화번호                    876850
k-팩스번호                    879348
k-세대타입(분양형태)              802824
k-관리방식                    876125
k-복도유형                    801214
k-난방방식                    800298
k-전체동수                    794232
k-전체세대수                   795327
k-건설사(시공사)                877637
k-시행사                     877834
k-사용검사일-사용승인일             876259
k-연면적                     795266
k-주거전용면적                  795327
k-관리비부과면적                 876125
k-전용면적별세대현황(60㎡이하)        876170
k-전용면적별세대현황(60㎡~85㎡이하)    876170
k-85㎡~135㎡이하              876170
k-수정일자    

In [99]:
concat_select['본번'] = concat_select['본번'].astype('str')
concat_select['부번'] = concat_select['부번'].astype('str')

continuous_columns = []
categorical_columns = []

for column in concat_select.columns:
    if pd.api.types.is_numeric_dtype(concat_select[column]):
        continuous_columns.append(column)
    else:
        categorical_columns.append(column)

print("연속형 변수:", continuous_columns)
print("범주형 변수:", categorical_columns)

연속형 변수: ['Unnamed: 0', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', 'k-전체동수', 'k-전체세대수', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', '건축면적', '주차대수', '좌표X', '좌표Y', 'target', '최상층', 'is_test']
범주형 변수: ['시군구', '번지', '본번', '부번', '아파트명', '도로명', 'k-단지분류(아파트,주상복합등등)', 'k-전화번호', 'k-팩스번호', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '단지신청일']


In [100]:
concat_select[categorical_columns] = concat_select[categorical_columns].fillna('NULL')

In [101]:
concat_select.isnull().sum()         # 결측치가 보간된 모습을 확인해봅니다.

Unnamed: 0                     0
시군구                            0
번지                             0
본번                             0
부번                             0
아파트명                           0
전용면적(㎡)                        0
계약년월                           0
계약일                            0
층                              0
건축년도                           0
도로명                            0
k-단지분류(아파트,주상복합등등)             0
k-전화번호                         0
k-팩스번호                         0
k-세대타입(분양형태)                   0
k-관리방식                         0
k-복도유형                         0
k-난방방식                         0
k-전체동수                    794232
k-전체세대수                   795327
k-건설사(시공사)                     0
k-시행사                          0
k-사용검사일-사용승인일                  0
k-연면적                     795266
k-주거전용면적                  795327
k-관리비부과면적                 876125
k-전용면적별세대현황(60㎡이하)        876170
k-전용면적별세대현황(60㎡~85㎡이하)    876170
k-85㎡~135㎡이하              876170
k-수정일자    

In [102]:
len(concat_select)

1128094

In [103]:
concat_select.reset_index(inplace=True)

In [104]:
concat_select.head(1)

,index,Unnamed: 0,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,...,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,최상층,is_test
0,0,0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,202307,26,...,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,NaN,5.0,1


In [105]:
concat_select_drop = concat_select.drop(["Unnamed: 0"], axis=1)

In [106]:
location_df = pd.read_csv("../../modified_data/final_data.csv")

In [107]:
location_df.head(1)

,index,아파트명,전용면적,계약일,층,건축년도,도로명,target,주소,좌표X,...,최대층수,지하철거리,구,동,계약년,계약월,아파트,도로,강남여부,신축여부
0,0,개포6차우성,79.97,8,3,1987,언주로 3,124000.0,서울특별시 강남구 개포동언주로 3,127.055926,...,5,1146.684379,강남구,개포동,2017,12,우성,언주로,1,0


In [108]:
right_filtered= ["index", "아파트명", "최대층수", "좌표X","좌표Y","station_area_0","station_area_1","실거래가격지수","면적구간","mean_price",'지하철거리',"구",'동',"계약년","계약월","아파트",'도로',"강남여부","신축여부"]

In [109]:
location_df_filtered = location_df[right_filtered]

In [110]:
location_df_filtered.head(2)

,index,아파트명,최대층수,좌표X,좌표Y,station_area_0,station_area_1,실거래가격지수,면적구간,mean_price,지하철거리,구,동,계약년,계약월,아파트,도로,강남여부,신축여부
0,0,개포6차우성,5,127.055926,37.476792,True,False,100.509309,3,84221.7277,1146.684379,강남구,개포동,2017,12,우성,언주로,1,0
1,1,개포6차우성,5,127.055926,37.476792,True,False,100.509309,3,84221.7277,1146.684379,강남구,개포동,2017,12,우성,언주로,1,0


In [111]:
concat_select_drop.head(10)

,index,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,...,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,최상층,is_test
0,0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.9700,202307,26,5,...,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,NaN,5.0,1
1,1,서울특별시 강남구 개포동,651-1,651.0,1.0,개포더샵트리에,108.2017,202308,15,10,...,의무,2022-02-23 13:01:10.0,Y,N,127.056394,37.484892,2022-02-23 11:05:05.0,NaN,16.0,1
2,2,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,161.0000,202307,28,15,...,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,NaN,15.0,1
3,3,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,133.4600,202308,10,14,...,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,NaN,15.0,1
4,4,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,104.4300,202308,18,6,...,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,NaN,15.0,1
5,5,서울특별시 강남구 개포동,187,187.0,0.0,개포주공5단지,74.2500,202307,28,8,...,의무,2015-03-02 11:08:24.0,Y,N,127.068028,37.487802,2013-03-07 09:46:42.0,NaN,14.0,1
6,6,서울특별시 강남구 개포동,185,185.0,0.0,개포주공6단지,83.2100,202307,2,11,...,의무,2018-03-19 10:55:24.0,Y,N,127.072682,37.489122,2013-03-07 09:46:38.0,NaN,15.0,1
7,7,서울특별시 강남구 개포동,185,185.0,0.0,개포주공6단지,73.0200,202308,14,12,...,의무,2018-03-19 10:55:24.0,Y,N,127.072682,37.489122,2013-03-07 09:46:38.0,NaN,15.0,1
8,8,서울특별시 강남구 개포동,185,185.0,0.0,개포주공6단지,53.0600,202308,24,9,...,의무,2018-03-19 10:55:24.0,Y,N,127.072682,37.489122,2013-03-07 09:46:38.0,NaN,15.0,1
9,9,서울특별시 강남구 개포동,1280,1280.0,0.0,래미안블레스티지,126.9280,202307,3,26,...,의무,2019-03-14 11:33:30.0,Y,N,127.064151,37.480049,2019-03-09 05:56:52.0,NaN,35.0,1


In [112]:
print(concat_select_drop.columns)
print(location_df_filtered.columns)


Index(['index', '시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층',
       '건축년도', '도로명', 'k-단지분류(아파트,주상복합등등)', 'k-전화번호', 'k-팩스번호', 'k-세대타입(분양형태)',
       'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)',
       'k-시행사', 'k-사용검사일-사용승인일', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적',
       'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하',
       'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태', '건축면적', '주차대수',
       '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '좌표X', '좌표Y',
       '단지신청일', 'target', '최상층', 'is_test'],
      dtype='object')
Index(['index', '아파트명', '최대층수', '좌표X', '좌표Y', 'station_area_0',
       'station_area_1', '실거래가격지수', '면적구간', 'mean_price', '지하철거리', '구', '동',
       '계약년', '계약월', '아파트', '도로', '강남여부', '신축여부'],
      dtype='object')


In [113]:
merge_with_location = pd.merge(location_df_filtered, concat_select_drop, on='index', how='left')


In [114]:
merge_with_location.head(3)

,index,아파트명_x,최대층수,좌표X_x,좌표Y_x,station_area_0,station_area_1,실거래가격지수,면적구간,mean_price,...,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X_y,좌표Y_y,단지신청일,target,최상층,is_test
0,0,개포6차우성,5,127.055926,37.476792,True,False,100.509309,3,84221.72770,...,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,NaN,5.0,1
1,1,개포6차우성,5,127.055926,37.476792,True,False,100.509309,3,84221.72770,...,의무,2022-02-23 13:01:10.0,Y,N,127.056394,37.484892,2022-02-23 11:05:05.0,NaN,16.0,1
2,2,개포6차우성,5,127.055926,37.476792,True,False,100.509309,2,86921.56087,...,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,NaN,15.0,1


In [115]:
merge_with_location.columns

Index(['index', '아파트명_x', '최대층수', '좌표X_x', '좌표Y_x', 'station_area_0',
       'station_area_1', '실거래가격지수', '면적구간', 'mean_price', '지하철거리', '구', '동',
       '계약년', '계약월', '아파트', '도로', '강남여부', '신축여부', '시군구', '번지', '본번', '부번',
       '아파트명_y', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', '도로명',
       'k-단지분류(아파트,주상복합등등)', 'k-전화번호', 'k-팩스번호', 'k-세대타입(분양형태)', 'k-관리방식',
       'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사',
       'k-사용검사일-사용승인일', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-수정일자', '고용보험관리번호',
       '경비비관리형태', '세대전기계약방법', '청소비관리형태', '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4',
       '단지승인일', '사용허가여부', '관리비 업로드', '좌표X_y', '좌표Y_y', '단지신청일', 'target',
       '최상층', 'is_test'],
      dtype='object')

In [116]:
pd.set_option('display.max_rows', None)


In [117]:
merge_with_location.loc[merge_with_location["최대층수"] < merge_with_location["최상층"], "최상층"] = merge_with_location["최상층"]

In [122]:
merge_with_location.columns

Index(['index', '아파트명_x', '최대층수', '좌표X_x', '좌표Y_x', 'station_area_0',
       'station_area_1', '실거래가격지수', '면적구간', 'mean_price', '지하철거리', '구', '동',
       '계약년', '계약월', '아파트', '도로', '강남여부', '신축여부', '시군구', '번지', '본번', '부번',
       '아파트명_y', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', '도로명',
       'k-단지분류(아파트,주상복합등등)', 'k-전화번호', 'k-팩스번호', 'k-세대타입(분양형태)', 'k-관리방식',
       'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사',
       'k-사용검사일-사용승인일', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-수정일자', '고용보험관리번호',
       '경비비관리형태', '세대전기계약방법', '청소비관리형태', '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4',
       '단지승인일', '사용허가여부', '관리비 업로드', '좌표X_y', '좌표Y_y', '단지신청일', 'target',
       '최상층', 'is_test'],
      dtype='object')

In [124]:
continuous_columns = []
categorical_columns = []

for column in merge_with_location.columns:
    if pd.api.types.is_numeric_dtype(merge_with_location[column]):
        continuous_columns.append(column)
    else:
        categorical_columns.append(column)

print("연속형 변수:", continuous_columns)
print("범주형 변수:", categorical_columns)

연속형 변수: ['index', '최대층수', '좌표X_x', '좌표Y_x', 'station_area_0', 'station_area_1', '실거래가격지수', '면적구간', 'mean_price', '지하철거리', '계약년', '계약월', '강남여부', '신축여부', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', 'k-전체동수', 'k-전체세대수', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', '건축면적', '주차대수', '좌표X_y', '좌표Y_y', 'target', '최상층', 'is_test']
범주형 변수: ['아파트명_x', '구', '동', '아파트', '도로', '시군구', '번지', '본번', '부번', '아파트명_y', '도로명', 'k-단지분류(아파트,주상복합등등)', 'k-전화번호', 'k-팩스번호', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '단지신청일']


In [131]:
merge_with_location.drop(['좌표X_y', '좌표Y_y'], axis=1, inplace=True)

In [132]:
merge_with_location['구'] = merge_with_location['시군구'].map(lambda x : x.split()[1])
merge_with_location['동'] = merge_with_location['시군구'].map(lambda x : x.split()[2])
del merge_with_location['시군구']

merge_with_location['계약년'] = merge_with_location['계약년월'].astype('str').map(lambda x : x[:4])
merge_with_location['계약월'] = merge_with_location['계약년월'].astype('str').map(lambda x : x[4:])
del merge_with_location['계약년월']

KeyError: '시군구'

In [127]:
all = list(concat_select['구'].unique())
gangnam = ['강서구', '영등포구', '동작구', '서초구', '강남구', '송파구', '강동구']
gangbuk = [x for x in all if x not in gangnam]

assert len(all) == len(gangnam) + len(gangbuk) 

In [133]:
merge_with_location.head(10)

,index,아파트명_x,최대층수,좌표X_x,좌표Y_x,station_area_0,station_area_1,실거래가격지수,면적구간,mean_price,...,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,단지신청일,target,최상층,is_test
0,0,개포6차우성,5,127.055926,37.476792,True,False,100.509309,3,84221.72770,...,4858.00,262.0,임의,2022-11-17 13:00:29.0,Y,N,2022-11-17 10:19:06.0,NaN,5.0,1
1,1,개포6차우성,5,127.055926,37.476792,True,False,100.509309,3,84221.72770,...,2724.46,305.0,의무,2022-02-23 13:01:10.0,Y,N,2022-02-23 11:05:05.0,NaN,16.0,1
2,2,개포6차우성,5,127.055926,37.476792,True,False,100.509309,2,86921.56087,...,61064.24,419.0,의무,1984-12-22 00:00:00.0,Y,N,2013-03-07 09:46:28.0,NaN,15.0,1
3,3,개포6차우성,5,127.055926,37.476792,True,False,102.018069,3,81495.96440,...,61064.24,419.0,의무,1984-12-22 00:00:00.0,Y,N,2013-03-07 09:46:28.0,NaN,15.0,1
4,4,개포6차우성,5,127.055926,37.476792,True,False,102.018069,3,81495.96440,...,61064.24,419.0,의무,1984-12-22 00:00:00.0,Y,N,2013-03-07 09:46:28.0,NaN,15.0,1
5,5,개포6차우성,5,127.055926,37.476792,True,False,102.018069,3,81495.96440,...,6524.00,652.0,의무,2015-03-02 11:08:24.0,Y,N,2013-03-07 09:46:42.0,NaN,14.0,1
6,6,개포6차우성,5,127.055926,37.476792,True,False,104.537846,3,69399.30821,...,0.00,1100.0,의무,2018-03-19 10:55:24.0,Y,N,2013-03-07 09:46:38.0,NaN,15.0,1
7,7,개포6차우성,5,127.055926,37.476792,True,False,105.500961,2,76850.76893,...,0.00,1100.0,의무,2018-03-19 10:55:24.0,Y,N,2013-03-07 09:46:38.0,NaN,15.0,1
8,8,개포6차우성,5,127.055926,37.476792,True,False,106.651247,3,72355.86702,...,0.00,1100.0,의무,2018-03-19 10:55:24.0,Y,N,2013-03-07 09:46:38.0,NaN,15.0,1
9,9,개포6차우성,5,127.055926,37.476792,True,False,108.041638,2,88110.66515,...,16155.05,3153.0,의무,2019-03-14 11:33:30.0,Y,N,2019-03-09 05:56:52.0,NaN,35.0,1


In [134]:
merge_with_location.columns

Index(['index', '아파트명_x', '최대층수', '좌표X_x', '좌표Y_x', 'station_area_0',
       'station_area_1', '실거래가격지수', '면적구간', 'mean_price', '지하철거리', '구', '동',
       '계약년', '계약월', '아파트', '도로', '강남여부', '신축여부', '번지', '본번', '부번', '아파트명_y',
       '전용면적(㎡)', '계약일', '층', '건축년도', '도로명', 'k-단지분류(아파트,주상복합등등)', 'k-전화번호',
       'k-팩스번호', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-전체동수',
       'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일', 'k-연면적', 'k-주거전용면적',
       'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)',
       'k-85㎡~135㎡이하', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '단지신청일', 'target', '최상층', 'is_test'],
      dtype='object')

In [135]:
merge_with_location['건축년도'].describe(percentiles = [0.1, 0.25, 0.5, 0.75, 0.8, 0.9])

count    1.101129e+06
mean     1.998841e+03
std      9.397126e+00
min      1.961000e+03
10%      1.986000e+03
25%      1.992000e+03
50%      2.000000e+03
75%      2.005000e+03
80%      2.006000e+03
90%      2.010000e+03
max      2.023000e+03
Name: 건축년도, dtype: float64

In [145]:
merge_with_location.drop(['아파트명_y'],axis=1, inplace=True)

In [146]:
dt_train = merge_with_location.query('is_test==0')
dt_test = merge_with_location.query('is_test==1')

# 이제 is_test 칼럼은 drop해줍니다.
dt_train.drop(['is_test'], axis = 1, inplace=True)
dt_test.drop(['is_test'], axis = 1, inplace=True)
print(dt_train.shape, dt_test.shape)

(1091857, 59) (9272, 59)


In [147]:
dt_test.head(1)

,index,아파트명_x,최대층수,좌표X_x,좌표Y_x,station_area_0,station_area_1,실거래가격지수,면적구간,mean_price,...,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,단지신청일,target,최상층
0,0,개포6차우성,5,127.055926,37.476792,True,False,100.509309,3,84221.7277,...,직영,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,2022-11-17 10:19:06.0,NaN,5.0


In [148]:
dt_test['target'] = 0

In [155]:
dt_test.isna().sum()

index                        0
아파트명_x                       0
최대층수                         0
좌표X_x                        0
좌표Y_x                        0
station_area_0               0
station_area_1               0
실거래가격지수                      0
면적구간                         0
mean_price                   0
지하철거리                        0
구                            0
동                            0
계약년                          0
계약월                          0
아파트                          0
도로                           0
강남여부                         0
신축여부                         0
번지                           0
본번                           0
부번                           0
전용면적(㎡)                      0
계약일                          0
층                            0
건축년도                         0
도로명                          0
k-단지분류(아파트,주상복합등등)           0
k-전화번호                       0
k-팩스번호                       0
k-세대타입(분양형태)                 0
k-관리방식                       0
k-복도유형  

In [149]:
continuous_columns_v2 = []
categorical_columns_v2 = []

for column in dt_train.columns:
    if pd.api.types.is_numeric_dtype(dt_train[column]):
        continuous_columns_v2.append(column)
    else:
        categorical_columns_v2.append(column)

print("연속형 변수:", continuous_columns_v2)
print("범주형 변수:", categorical_columns_v2)

연속형 변수: ['index', '최대층수', '좌표X_x', '좌표Y_x', 'station_area_0', 'station_area_1', '실거래가격지수', '면적구간', 'mean_price', '지하철거리', '강남여부', '신축여부', '전용면적(㎡)', '계약일', '층', '건축년도', 'k-전체동수', 'k-전체세대수', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', '건축면적', '주차대수', 'target', '최상층']
범주형 변수: ['아파트명_x', '구', '동', '계약년', '계약월', '아파트', '도로', '번지', '본번', '부번', '도로명', 'k-단지분류(아파트,주상복합등등)', 'k-전화번호', 'k-팩스번호', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '단지신청일']


In [151]:
label_encoders = {}
for col in tqdm(categorical_columns_v2):
    lbl = LabelEncoder()

    # 훈련 데이터로 Label-Encoding을 fit
    lbl.fit(dt_train[col].astype(str))
    dt_train[col] = lbl.transform(dt_train[col].astype(str))
    
    # 레이블 인코더 저장
    label_encoders[col] = lbl

    # 테스트 데이터에서 훈련 데이터에 없는 새로운 레이블을 처리합니다.
    # unseen label을 위한 후처리
    unseen_labels = [label for label in dt_test[col].astype(str).unique() if label not in lbl.classes_]
    if unseen_labels:
        lbl.classes_ = np.append(lbl.classes_, unseen_labels)
    
    # 테스트 데이터 변환
    dt_test[col] = lbl.transform(dt_test[col].astype(str))

100%|██████████| 31/31 [00:23<00:00,  1.31it/s]


In [152]:
assert dt_train.shape[1] == dt_test.shape[1]  

In [153]:
# Target과 독립변수들을 분리해줍니다.
y_train = dt_train['target']
X_train = dt_train.drop(['target'], axis=1)

# Hold out split을 사용해 학습 데이터와 검증 데이터를 8:2 비율로 나누겠습니다.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2023)

In [154]:
model = RandomForestRegressor(n_estimators=5, criterion='squared_error', random_state=1, n_jobs=-1)
model.fit(X_train, y_train)
pred = model.predict(X_val)

ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values